In [1]:
# International Bonds - BNDX, Bloomberg Barclays Global Aggregate ex-USD Float Adjusted RIC Capped Index (USD Hedged): 
#                       Using FTSE World Broad Investment-Grade Bond Index (WorldBIG) instead

In [2]:
import numpy as np
import pandas as pd
from PyPDF2 import PdfFileReader
import re
import datetime
import requests
import os

In [3]:
bndxURL = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20191031&IsManual=true"

In [4]:
year = ''
month = ''
day = ''

In [5]:
cur_year = datetime.datetime.now().year
cur_year

2020

In [6]:
error_url = 'https://www.ftserussell.com/products/indices/home/errorview?aspxerrorpath=/Analytics/FactSheets/Home/DownloadSingleIssueByDate'

In [7]:
quarterly_links = []

In [8]:
# Quarterly
for y in range(2018,cur_year+1):
    year = str(y)
    for m in range(3,13,3):
        if m < 10:
            month = '0'+ str(m)
        else:
            month = str(m)
        for d in range(28,32):
            day = str(d)
            date = year + month + day
            status = 'true'
            url = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate="+date+"&IsManual="+status+".pdf"
            response = requests.get(url)
            if response.url == error_url:
                status = 'false'
                url = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate="+date+"&IsManual="+status+".pdf"
                response = requests.get(url)
                if response.url == error_url:
                    print(date + " Failed")
            elif response.url != error_url:
                print(url)
                quarterly_links.append(url)
                break

https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180328&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180628&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180928&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20181228&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190328&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190628&IsManual=true.pdf
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190928&IsManual=true.pdf
https://research.ftserussell.com/Analytic

In [9]:
# Downloading files
for url in quarterly_links: 
    file_name = url.split('/')[-1]    
    print("Downloading file:%s"%file_name) 
    r = requests.get(url, stream = True) 
    with open(file_name, 'wb') as f: 
        for chunk in r.iter_content(chunk_size = 1024*1024): 
            if chunk: 
                f.write(chunk) 
    print("%s downloaded!\n"%file_name)
print("All reports downloaded!")

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180328&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180628&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20180928&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20181228&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190328&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190628&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20190928&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20191228&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20200328&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20200628&IsManual=true.pdf downloaded!

DownloadSingleIssueByDate?IssueName=WBIG&IssueDate=20200928&IsManual=true.pdf downloaded!


In [10]:
file_names = []
for url in quarterly_links:
    file = url.split('/')[-1]
    file_names.append(file)

In [11]:
bndxDF = pd.DataFrame(columns=['File','MC'])
for x in range(len(file_names)):
    pdf = PdfFileReader(file_names[x])
    pdf_page = pdf.getPage(1)
    pdf_text = pdf_page.extractText()
    pdf_text = pdf_text.replace('\n',' ')
    pdf_text = pdf_text.replace('  ',' ')
    pdf_text = pdf_text.replace('  ',' ')
    mc = re.findall('Totals \d+ \d+,\d+,\d+', pdf_text, re.IGNORECASE)
    if mc == []:
        mc = 'n/a'
        row = [file_names[x],mc]
        bndxDF.loc[x] = row
        continue
    else:
        mc_1 = mc[0].replace(',','')
        mc_2 = mc_1.replace('\n','')
        mc_3 = mc_2.replace('Totals','')
        mc = [int(s) for s in mc_3.split() if s.isdigit()][1]
        row = [file_names[x],mc]
        bndxDF.loc[x] = row

PdfReadError: EOF marker not found

In [ ]:
bndxDF

In [ ]:
# Deleting files
for url in quarterly_links:
    file = url.split('/')[-1]
    print("Deleting file:%s"%file) 
    os.remove(file)
    print("%s deleted!\n"%file)
print("All reports deleted!")